In [4]:
from data_utils import LMOrderedIterator
import torch

In [89]:
it = LMOrderedIterator(torch.tensor(range(100)), bsz=3, bptt=5)
print(it.data.shape)
print(it.data[0])

torch.Size([33, 3])
tensor([ 0, 33, 66])


In [90]:
it.data.view(-1).shape

torch.Size([99])

In [24]:
data, target, seq_len = it.get_batch(0)
print(data.shape, seq_len)
print(data[:, 0])
print(target[:, 0])

torch.Size([5, 3]) 5
tensor([0, 1, 2, 3, 4])
tensor([1, 2, 3, 4, 5])


In [107]:
# What do consecutive batches look like?
def test_print(i):
    for data, target, seq in i:
        # First and last item of each batch
        print(data[:,0], '\t', data[:,-1])
test_print(iter(it))
print(len(list(iter(it))))

tensor([0, 1, 2, 3, 4]) 	 tensor([66, 67, 68, 69, 70])
tensor([5, 6, 7, 8, 9]) 	 tensor([71, 72, 73, 74, 75])
tensor([10, 11, 12, 13, 14]) 	 tensor([76, 77, 78, 79, 80])
tensor([15, 16, 17, 18, 19]) 	 tensor([81, 82, 83, 84, 85])
tensor([20, 21, 22, 23, 24]) 	 tensor([86, 87, 88, 89, 90])
tensor([25, 26, 27, 28, 29]) 	 tensor([91, 92, 93, 94, 95])
tensor([30, 31]) 	 tensor([96, 97])
7


In [105]:
def test_dist(it):
    for i in range(2):
        print(f'** rank {i} **')
        test_print(it.get_dist_iter(rank=i, max_rank=2))
        print(len(list(it.get_dist_iter(rank=i, max_rank=2))))
test_dist(it)

** rank 0 **
tensor([0, 1, 2, 3, 4]) 	 tensor([66, 67, 68, 69, 70])
tensor([5, 6, 7, 8, 9]) 	 tensor([71, 72, 73, 74, 75])
tensor([10, 11, 12, 13, 14]) 	 tensor([76, 77, 78, 79, 80])
tensor([15, 16, 17, 18, 19]) 	 tensor([81, 82, 83, 84, 85])
4
** rank 1 **
tensor([16, 17, 18, 19, 20]) 	 tensor([82, 83, 84, 85, 86])
tensor([21, 22, 23, 24, 25]) 	 tensor([87, 88, 89, 90, 91])
tensor([26, 27, 28, 29, 30]) 	 tensor([92, 93, 94, 95, 96])
tensor([31]) 	 tensor([97])
4


In [81]:
#  for i in range(start, self.data.size(0) - 1, self.bptt):
#      yield self.get_batch(i)

def get_dist_iter(self, rank, max_rank, start=0):
    chunk_len = self.data.size(0) // max_rank

    for i in range(start, chunk_len, self.bptt):
        yield self.get_batch(i + (rank * chunk_len))

def test_dist(it, f):
    for i in range(2):
        print(f'** rank {i} **')
        test_print(f(it, rank=i, max_rank=2))
test_dist(it, get_dist_iter)

** rank 0 **
tensor([0, 1, 2, 3, 4]) 	 tensor([66, 67, 68, 69, 70])
tensor([5, 6, 7, 8, 9]) 	 tensor([71, 72, 73, 74, 75])
tensor([10, 11, 12, 13, 14]) 	 tensor([76, 77, 78, 79, 80])
tensor([15, 16, 17, 18, 19]) 	 tensor([81, 82, 83, 84, 85])
** rank 1 **
tensor([16, 17, 18, 19, 20]) 	 tensor([82, 83, 84, 85, 86])
tensor([21, 22, 23, 24, 25]) 	 tensor([87, 88, 89, 90, 91])
tensor([26, 27, 28, 29, 30]) 	 tensor([92, 93, 94, 95, 96])
tensor([31]) 	 tensor([97])


In [85]:
its = (LMOrderedIterator(torch.tensor(range(50)), bsz=3, bptt=5),
    LMOrderedIterator(torch.tensor(range(50,100)), bsz=3, bptt=5))
for i, it in enumerate(its):
    print(i)
    test_print(it)

0
tensor([0, 1, 2, 3, 4]) 	 tensor([32, 33, 34, 35, 36])
tensor([5, 6, 7, 8, 9]) 	 tensor([37, 38, 39, 40, 41])
tensor([10, 11, 12, 13, 14]) 	 tensor([42, 43, 44, 45, 46])
1
tensor([50, 51, 52, 53, 54]) 	 tensor([82, 83, 84, 85, 86])
tensor([55, 56, 57, 58, 59]) 	 tensor([87, 88, 89, 90, 91])
tensor([60, 61, 62, 63, 64]) 	 tensor([92, 93, 94, 95, 96])


In [93]:
class O:
    def __init__(self, a):
        self.a = a
        
a = O(1)
a.__getattribute__('a')

1

In [101]:
from data_utils import Corpus
class FakeCorpus(Corpus):
    def __init__(self, data):
        self.train = torch.tensor(data)
        self.valid = torch.tensor(data)
        self.test = torch.tensor(data)
        self.dataset='wt103'
    
    def get_dist_iterator(self, split, rank, max_rank, *args, **kwargs):
        if self.dataset == 'lm1b':
            raise NotImplementedError('See get_iterator')
        data = self.__getattribute__(split)
        chunk_size = data.size(0) // max_rank
        return LMOrderedIterator(data[rank * chunk_size:(rank+1) * chunk_size], *args, **kwargs)

a = FakeCorpus(range(100))

In [97]:
test_print(a.get_iterator(split='train', bsz=3, bptt=5))

tensor([0, 1, 2, 3, 4]) 	 tensor([66, 67, 68, 69, 70])
tensor([5, 6, 7, 8, 9]) 	 tensor([71, 72, 73, 74, 75])
tensor([10, 11, 12, 13, 14]) 	 tensor([76, 77, 78, 79, 80])
tensor([15, 16, 17, 18, 19]) 	 tensor([81, 82, 83, 84, 85])
tensor([20, 21, 22, 23, 24]) 	 tensor([86, 87, 88, 89, 90])
tensor([25, 26, 27, 28, 29]) 	 tensor([91, 92, 93, 94, 95])
tensor([30, 31]) 	 tensor([96, 97])


In [103]:
for i in range(2):
    print(i)
    test_print(a.get_dist_iterator(split='train', rank=i, max_rank=2, bsz=3, bptt=5))

0
tensor([0, 1, 2, 3, 4]) 	 tensor([32, 33, 34, 35, 36])
tensor([5, 6, 7, 8, 9]) 	 tensor([37, 38, 39, 40, 41])
tensor([10, 11, 12, 13, 14]) 	 tensor([42, 43, 44, 45, 46])
1
tensor([50, 51, 52, 53, 54]) 	 tensor([82, 83, 84, 85, 86])
tensor([55, 56, 57, 58, 59]) 	 tensor([87, 88, 89, 90, 91])
tensor([60, 61, 62, 63, 64]) 	 tensor([92, 93, 94, 95, 96])
